In [ ]:
from datascience import *
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

# Functions

In [ ]:
def spread(values):
    spread_val = max(values) - min(values)
    return spread_val

**Question:** What is the next function doing?

In [ ]:
def f(s):
    return np.round(s / sum(s) * 100, 2)

In [ ]:
f(make_array(1,2,3,4))

In [ ]:
s = make_array(1,2,3,4)
sum(s)

In [ ]:
s / 10

In [ ]:
spread(s)

In [ ]:
abs(-3), abs(4.5)

In [ ]:
def convert_to_percents(s):
    return np.round(s / sum(s) * 100, 2)

In [ ]:
convert_to_percents(make_array(2,4,6,8))

In [ ]:
# make a function that doubles a number
def double(x):
    return 2 * x

In [ ]:
double(2)

In [ ]:
y = 5
double(y)

In [ ]:
y
double_y = double(y)
double_y, y

In [ ]:
# make a function that doubles a number
def double(x):
    """
    Returns twice the input variable
    """
    return 2 * x

*Make sure to add doc strings*
#### Help(function)

In [ ]:
help(double)

#### Scope
Does the value of a name change?

In [ ]:
y = 10
double(y / 10)

What is the value of y?

#### Multiple Arguments & Default Arguments

In [ ]:
def double_and_multiply(arg1, arg2):
    """
    Computes and returns the value of 2*arg1 * 2*arg2
    """
    return double(arg1) * double(arg2)

In [ ]:
double_and_multiply(3, 6)

In [ ]:
def double_and_multiply2(arg1, arg2=1):
    """
    Computes and returns the value of 2*arg1 * 2*arg2.
    arg2 defaults to 1
    """


In [ ]:
# answer below

In [ ]:
# answer below

In [ ]:
return double(arg1) * double(arg2)

In [ ]:
double_and_multiply2(5), double_and_multiply2(5,1)

back to slides

# `Apply`ing functions to Tables

In [ ]:
def cut_off_at_a_billion(x):
    '''
    Returns x if x is less than 1e9, otherwise returns 1e9
    '''
    if x > 1e9:
        return 1e9
    return x

In [ ]:
cut_off_at_a_billion(1)

In [ ]:
cut_off_at_a_billion(1e10)

In [ ]:
top_movies = Table.read_table('top_movies_2017.csv')
top_movies

### Apply billion cut off function to a Column and add it to the table

In [ ]:
gross_adjusted_capped = top_movies.apply(cut_off_at_a_billion, 'Gross (Adjusted)')
top_movies = top_movies.with_column('gross_adjusted_capped', gross_adjusted_capped)

In [ ]:
gross_capped = top_movies.apply(cut_off_at_a_billion, 'Gross')
top_movies = top_movies.with_column('gross_capped', gross_capped)
top_movies

### Heights example

In [ ]:
galton = Table.read_table('galton.csv')
galton

In [ ]:
galton.num_rows

In [ ]:
## We only want father, mother, and child heights
## How do we get that

In [ ]:
height = galton.select('father', 'mother', 'childHeight')
height

#### Can we guess height of a child given the parents' height?
What type of plot will show us if there is a correlation?

In [ ]:
height.labels

In [ ]:
height.scatter('childHeight')

#### What do we see in this chart?

In [ ]:
# let's use average heights from the galton table
# scatter plot with parent average on x-axis and child height on y-axis
galton.scatter('midparentHeight', 'childHeight')

In [ ]:
galton.scatter('midparentHeight', 'childHeight')
plots.plot([67.5, 67.5], [50, 85], color='red', lw=2)
plots.plot([68.5, 68.5], [50, 85], color='red', lw=2);

In [ ]:
# let's compute the mean height of children in this range
nearby = galton.where('midparentHeight', are.between(67.5, 68.5))
nearby_mean = nearby.column('childHeight').mean()
nearby_mean

In [ ]:
# Lets schow the mean child height 
galton.scatter('midparentHeight', 'childHeight')
plots.plot([67.5, 67.5], [50, 85], color='red', lw=2)
plots.plot([68.5, 68.5], [50, 85], color='red', lw=2)
plots.scatter(68, nearby_mean, color='red', s=50);

In [ ]:
# Now let's right a function to make this prediction in general

In [ ]:
def predict_height(parentsMidHeight):
    nearby = galton.where('midparentHeight', are.between(parentsMidHeight - 0.5, parentsMidHeight + .5))
    nearby_mean = nearby.column('childHeight').mean()
    return nearby_mean

In [ ]:
predict_height(68)

In [ ]:
predict_height(74)

In [ ]:
def predict(h):
    nearby = galton.where('midparentHeight', are.between(h - 1/2, h + 1/2))
    return nearby.column('childHeight').mean()

In [ ]:
# Now lets test this function on some examples

In [ ]:
# Now can we make this prediction on all our examples?

In [ ]:
galton.apply(predict, 'midparentHeight')

In [ ]:
# let's add the predictions to our galton table

In [ ]:
galton = galton.with_column("predictedHeight",  galton.apply(predict, 'midparentHeight'))

In [ ]:
galton.select(
    'midparentHeight', 'childHeight', 'predictedHeight').scatter('midparentHeight')

### Prediction Accuracy

In [ ]:
# Let's write a function to predict the difference between 
# the prediction and the true value

In [ ]:
galton.labels

In [ ]:
galton = galton.drop('child_prediction', 'predictHeight')

In [ ]:
galton = galton.with_column('predictionError', abs(galton.column('predictedHeight') - galton.column('childHeight')))

### What type of plot should we use to show the errors

In [ ]:
galton.hist('predictionError')

In [ ]:
galton.hist('predictionError', group='gender', )

back to slides

# Group Multiple Columns

In [ ]:
sky = Table.read_table('skyscrapers_v2.csv')
sky = (sky.with_column('age', 2020 - sky.column('completed'))
          .drop('completed'))
sky.show(3)

#### 1. For each city, what’s the tallest building for each material?

In [ ]:
sky.drop('age').group(['city', 'material'], max)

back to slides

# Pivot

#### 1. For each city, what’s the tallest building for each material?
Use pivot here!


In [ ]:
sky_pivot = sky.pivot('material', 'city', 'height', max)
sky_pivot.show()

#### 2. For each city, what’s the height difference between the tallest steel building and the tallest concrete building?

Use sky_pivot

In [ ]:
sky_pivot = sky_pivot.with_column( 'difference', abs(sky_pivot.column('steel') - sky_pivot.column('concrete')) ) 
sky_pivot

(back to slides)
# Joins

In [ ]:
billionaires = Table.read_table('billionaires.csv')
billionaires

In [ ]:
countries = Table.read_table('countries.csv')
countries

### What can we join these two tables on?

##### How many billionaries are in a specific industry and Continent?

In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook()
grader.export("Lecture_7_completed.ipynb", pdf=False)